<a href="https://colab.research.google.com/github/KhangTran2503/AI/blob/master/Policy_Iteration_vs_Value_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Kết quả:**
>  Trong toy games FrozenLakev0 và FrozeLake8x8-v0 thì Policy Iteration chạy nhanh hơn.
>  Còn trong Taxi-v3 thì Value Iteration chạy nhanh hơn.


###**Nhận xét:**
> 1. Policy Iteration cài đặt phức tạp hơn Value Iteration.
> 2. Policy Iteration thường hội tụ nhanh hơn nhưng lại phức tạp về mặt tính toán.
> 3. Policy Iteration dựa vào optimality Bellman operator
  còn Value Iteration thì là  Bellman operator.

###**Tham Khảo:**
1. [What is difference between value iteration and policy iteration?](https://stackoverflow.com/questions/37370015/what-is-the-difference-between-value-iteration-and-policy-iteration)
2. [Value Iteration](http://incompleteideas.net/book/first/ebook/node44.html)
3. [Policy Iteration](http://incompleteideas.net/book/first/ebook/node43.html)



In [0]:
import gym
import numpy as np
import time

### **Policy Iteration:**

In [0]:
class Policy():
    def __init__(self, name: str, max_iters, gamma):
        self.env = gym.make(name)
        self.max_iters = max_iters
        self.gamma = gamma
        self.name = name 
    
    def policy_evaluation(self, policy):
    
        # initialize value table with zeros
        v_values = np.zeros(self.env.observation_space.n)
        while True:
        
            # store prev_v_values 
            prev_v_values = np.copy(v_values)

            # for each state, compute the value according to the policy
            for state in range(self.env.observation_space.n):
                action = policy[state]
                q_value = 0
                for prob, next_state, reward, done in self.env.P[state][action]:
                    q_value += prob* (reward + self.gamma * prev_v_values[next_state])

                v_values[state] = q_value
            
            # check convergence
            if np.all(np.isclose(v_values, prev_v_values)):
                break
            
        return v_values

    def policy_extraction(self, v_values):
 
        # Initialize the policy with zeros
        policy = np.zeros(self.env.observation_space.n, dtype=np.int) 
    
        for state in range(self.env.observation_space.n):
        
            # initialize the q_values for a state
            q_values = np.zeros(self.env.action_space.n)
        
            # compute q_value for all ations in the state
            for action in range(self.env.action_space.n):
                q_value = 0
                for prob, next_state, reward, done in self.env.P[state][action]:
                    q_value += prob* (reward + self.gamma * v_values[next_state])
                q_values[action] = q_value
            # Select the best action
            policy[state] = np.argmax(q_values)
    
        return policy 

    def policy_iteration(self):
    
        # Initialize policy with zeros
        #old_policy = np.zeros(env.observation_space.n)
        old_policy = np.random.choice(self.env.action_space.n, size=(self.env.observation_space.n))
    
        for i in range(self.max_iters):
        
            # compute new_value from policy_evaluation function
            new_value = self.policy_evaluation(old_policy)
        
            # Extract new policy from policy_extraction function
            new_policy = self.policy_extraction(new_value)

            if (np.all(old_policy == new_policy)):
                print('Policy-iteration coverged at {}-th iteration.'.format(i+1))
                break
            old_policy = new_policy
        
        return new_policy
    
    def play(self, policy):
        state = self.env.reset()
        steps = 0
        done = False
        while not done:
            action = policy[state]
            next_state, reward, done, info = self.env.step(action)
            steps += 1
            state = next_state
    
        return (reward, steps)

    def isGoal(self,reward):
        if self.name == 'Taxi-v3': return (reward > 0)
        else: return (reward == 1)
        
    def play_multiple_times(self,policy):
        num_episodes = 1000
        list_of_steps = []
        num_failures = 0
        temp = 0
    
        for i in range(num_episodes):
            reward, steps = self.play(policy)
            if self.isGoal(reward):
               list_of_steps.append(steps)
            else:
                num_failures += 1

        print('# failures: {}/{}'.format(num_failures, num_episodes))
        print('avg. # steps: {}'.format(np.mean(list_of_steps)))


### **Value Iteration:**

In [0]:
class Value():
    def __init__(self, name: str, max_iters, gamma):
        self.env = gym.make(name)
        self.max_iters = max_iters
        self.gamma = gamma
        self.name = name 

    def optimal_value(self):
        v_values = np.zeros(self.env.observation_space.n)
        for i in range(self.max_iters):
            prev_v_values = np.copy(v_values)

            # Compute value for each state
            for state in range(self.env.observation_space.n):
                q_values = []

                # Compute q-value for each action
                for action in range(self.env.action_space.n):                
                    q_value = 0
                    for prob, next_state, reward, done in self.env.P[state][action]:
                        q_value += prob * (reward + self.gamma * prev_v_values[next_state])
                    q_values.append(q_value)
                
                # Select the best action
                best_action = np.argmax(np.asarray(q_values))
                v_values[state] = q_values[best_action]
            
            # Check convergence
            if np.all(np.isclose(v_values, prev_v_values)):
                print('Value-iteration converged at {}-th iteration.'.format(i))
                break
        
        return v_values

    def value_iteration(self):
        v_values = self.optimal_value()
        policy = np.zeros(self.env.observation_space.n, dtype=np.int)
        
        # Compute the best action for each state in the game
        # Compute q-values for each (state-action) pair in the game
        for state in range(self.env.observation_space.n):
            q_values = []

            # Compute q-values for each action
            for action in range(self.env.action_space.n):
                q_value = 0
                for prob, next_state, reward, done in self.env.P[state][action]:
                    q_value += prob * (reward + self.gamma * v_values[next_state])
                q_values.append(q_value)

            # Select the best action
            best_action = np.argmax(np.asarray(q_values))
            policy[state] = best_action
        
        return policy

    def play(self, policy):
        state = self.env.reset()
        steps = 0
        done = False
        while not done:
            action = policy[state]
            next_state, reward, done, info = self.env.step(action)
            steps += 1
            state = next_state
    
        return (reward, steps)

    def isGoal(self,reward):
        if self.name == 'Taxi-v3': return (reward > 0)
        else: return (reward == 1)

    def play_multiple_times(self,policy):
        num_episodes = 1000
        list_of_steps = []
        num_failures = 0
        temp = 0
    
        for i in range(num_episodes):
            reward, steps = self.play(policy)
            if self.isGoal(reward):
               list_of_steps.append(steps)
            else:
                num_failures += 1

        print('# failures: {}/{}'.format(num_failures, num_episodes))
        print('avg. # steps: {}'.format(np.mean(list_of_steps)))


####**FrozenLakev0 :**

**1. Policy Iteration:**

In [20]:
#start time 
start_time = time.time()

#Run Game
FrozenLake_v0 = Policy('FrozenLake-v0', max_iters=1000, gamma=0.9)
policy = FrozenLake_v0.policy_iteration()

#End time
end_time = time.time()

#Total time
print("Execute time Policy_iteration: {}".format(end_time-start_time))

#Avg
FrozenLake_v0.play_multiple_times(policy)

Policy-iteration coverged at 3-th iteration.
Execute time Policy_iteration: 0.0345458984375
# failures: 283/1000
avg. # steps: 36.38772663877266


**2.Value Iteration:**

In [22]:
#start time
start_time = time.time()

# Run Game
FrozenLake_v0 = Value('FrozenLake-v0', max_iters=1000, gamma=0.9)
policy = FrozenLake_v0.value_iteration()

#End time
end_time = time.time()

#Total time
print("Execute time Value_iteration: {}".format(end_time-start_time))

#Avg
FrozenLake_v0.play_multiple_times(policy)

Value-iteration converged at 79-th iteration.
Execute time Value_iteration: 0.04042673110961914
# failures: 258/1000
avg. # steps: 37.288409703504044


####**FrozenLake8x8-v0 :**

**1. Policy Iteration:**

In [28]:
#start time 
start_time = time.time()

#Run Game
FrozenLake8x8_v0 = Policy('FrozenLake8x8-v0', max_iters=1000, gamma=0.9)
policy = FrozenLake8x8_v0.policy_iteration()

#End time
end_time = time.time()

#Total time
print("Execute time Policy_iteration: {}".format(end_time-start_time))

#Avg
FrozenLake8x8_v0.play_multiple_times(policy)

Policy-iteration coverged at 8-th iteration.
Execute time Policy_iteration: 0.216172456741333
# failures: 250/1000
avg. # steps: 70.66533333333334


**2.Value Iteration:**

In [29]:
#start time
start_time = time.time()

# Run Game
FrozenLake8x8_v0 = Value('FrozenLake8x8-v0', max_iters=1000, gamma=0.9)
policy = FrozenLake8x8_v0.value_iteration()

#End time
end_time = time.time()

#Total time
print("Execute time Value_iteration: {}".format(end_time-start_time))

#Avg
FrozenLake8x8_v0.play_multiple_times(policy)

Value-iteration converged at 117-th iteration.
Execute time Value_iteration: 0.27852559089660645
# failures: 257/1000
avg. # steps: 74.36339165545087


####**Taxi-v3:**

**1. Policy Iteration:**

In [31]:
#start time 
start_time = time.time()

#Run Game
Taxi_v3 = Policy('Taxi-v3', max_iters=1000, gamma=0.9)
policy = Taxi_v3.policy_iteration()

#End time
end_time = time.time()

#Total time
print("Execute time Policy_iteration: {}".format(end_time-start_time))

#Avg
Taxi_v3.play_multiple_times(policy)

Policy-iteration coverged at 18-th iteration.
Execute time Policy_iteration: 2.701164722442627
# failures: 0/1000
avg. # steps: 12.996


**2.Value Iteration:**

In [32]:
#start time
start_time = time.time()

# Run Game
Taxi_v3 = Value('Taxi-v3', max_iters=1000, gamma=0.9)
policy = Taxi_v3.value_iteration()

#End time
end_time = time.time()

#Total time
print("Execute time Value_iteration: {}".format(end_time-start_time))

#Avg
Taxi_v3.play_multiple_times(policy)

Value-iteration converged at 116-th iteration.
Execute time Value_iteration: 0.9904379844665527
# failures: 0/1000
avg. # steps: 13.234
